IMPORT

You should have **Voxelizer.py** in your pyhton working folder (download it here https://github.com/MirkoZanon/Voxelizer).

In [1]:
import numan as nu
import vodex as vx
import Voxelizer as voxz
import pandas as pd
import numpy as np
import os

import matplotlib
import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter
import seaborn as sns
from tqdm.notebook import tqdm, trange

FOLDERS AND DATA

The program assumes a main working folder (to set here, the program will then work inside this main folder) 
inside which you should have a **'*mask.tif*' file** with a mask of ROIs defined by integers and a **'*/processed*' folder**.
This 'processed' folder is previously created by numan and should contain an additional **'*/drift_corrected_af_to1000*'** folder with the truncated drift corrected files and their relative annotation (number+shape+spread labels) saved as **'*stimuli_truncated_timelines.csv*'**.
If these hierachies and names are not matching you have to modify them in the code below (or rename your original files)!

The process will create (if not existing) a folder inside the main path called '*/voxelizer_final_datasets*', where it will output a series of csv files for each ROI of the mask (file output name code: fishInfo_ROI#.csv)

In [10]:
# raw dataset folder and info
base_folder = "/Users/mirckuz/Desktop/20230601_Hz09_casper_h2bcamp7f_7dpf_60Z_1hzvol_2P_1v2v3v4v5_processed"
fishInfo = 'Hz09_casper_h2bcamp7f_7dpf'

# parameters for analysis
superVoxel_size = [3,3,3] # dimention of segmentation 'super voxel' in voxels. IMPORTANT, coordinates order: [z,y,x]
ROIs_to_analyze = [1,2,3,4,5] # define the ROIs you want to analyze (the ROI ID is the integer number relative to your mask)
truncated_stimulus_chunck_dimention = 9 # number of volumes in truncated window (remember python starts counting from zero)
normalization_frames = [0,1,2] # frames in truncated window to consider as F0 background for dF/F0 normalization
frames_per_volume = 60 # orginal frame per volume in raw numan experiment
starting_slice = 0 # orginal starting slice in raw numan experiment
n_vol_batch = 8 # number of volumes to load at a time for batch analysis

print('Note: you declared that your dataset is cut in \033[1mchunks of '+str(truncated_stimulus_chunck_dimention)+' volumes\033[0m. This info should be fulfilled by data in /drift_corrected_af_to1000 for a correct analysis!')

Note: you declared that your dataset is cut in chunks of 9 volumes. This info should be fulfilled by data in /drift_corrected_af_to1000 for a correct analysis!


Ready to go! Run super voxel analysis analysis - this takes some time depending on recording and ROI size! (Should be approx 30min for an avareage ROI size and standard truncated exp)

In [12]:
os.chdir(base_folder) # set working directory to main raw data folder defined above
mask_file = "mask.tif"
annotations = pd.read_csv('./processed/stimuli_truncated_timelines.csv')
print('Loading experiment...')
experiment = vx.Experiment.from_dir('./processed/drift_corrected_af_to1000',frames_per_volume,starting_slice,verbose=False)
experiment.add_annotations_from_volume_annotation_df(annotations)

for roi in ROIs_to_analyze:
    print('\033[1m\nPROCESSING ROI '+str(roi)+'\033[0m\n')

    print('Voxelizing ROI '+str(roi)+'... This could take time depending on your data and ROI size, but approx less than 20 min')
    voxelizer = voxz.Voxelizer(mask_file, superVoxel_size, roi)
    table = voxelizer.process_movie(experiment, n_vol_batch)

    print('Normalizing signal in ROI '+str(roi)+'...')
    df = voxelizer.create_normalized_signal_df(table,truncated_stimulus_chunck_dimention,normalization_frames)
    signals = np.array(df).T
    
    print('Preparing data table for ROI '+str(roi)+'...')
    stim_volumes = experiment.choose_volumes([("number","d1"), ("number","d2"), ("number","d3"),("number","d4"), ("number","d5")], logic = "or")
    stim_signal1=signals[:,stim_volumes]
    stim_signal2=signals[:,[x + 1 for x in stim_volumes]]
    stim_signal3=signals[:,[x + 2 for x in stim_volumes]]
    stim_signal = (stim_signal1+stim_signal2+stim_signal3)/3
    print(' You obtained -> SuperVoxels X trials: ' + str(stim_signal.shape)+'\n')
    
    annotation_dict2= {f"cell_{ic}": stim_signal[ic] for ic in np.arange(len(signals))}
    annotation_dict=experiment.get_volume_annotations(stim_volumes)
    annotation_dict.update(annotation_dict2)
    annotation_df=pd.DataFrame(annotation_dict)

    #calculate control trials label array
    C_pd = pd.factorize((annotation_df['shape']+ annotation_df['spread']), sort=True)
    #C_pd = pd.factorize((annotation_df['shape']), sort=True)
    labelC = C_pd[1]
    C = np.array(C_pd[0])
    print('Control conditions label array, shape -> (trials,): ' + str(C.shape))

    #calculate stimulus trials label array
    Q_pd = pd.factorize(annotation_df['number'], sort=True)
    labelQ = Q_pd[1]
    Q = np.array(Q_pd[0])
    print('Stimulus label array, shape -> (trials,): ' + str(Q.shape))

    Hf = np.array(annotation_df.iloc[:, 4:annotation_df.shape[1]])
    print('Final dataset ROI '+str(roi)+', shape -> (trials,cells): ' + str(Hf.shape))

    # merge the three array of interest Hf(data), Q(condition), C(control)
    final_labels = np.stack((C,Q),axis=1)
    total_df = np.concatenate((final_labels, Hf),axis=1)

    # save at /voxelizer_final_datasets
    if not os.path.exists('./voxelizer_final_datasets'):
        os.makedirs('./voxelizer_final_datasets')
    np.savetxt('./voxelizer_final_datasets/'+fishInfo+'_ROI'+str(roi)+'.csv', total_df, delimiter=',')
    print('!FINAL ARRAY FOR ROI '+str(roi)+' SAVED AS CSV!, shape -> (trials,2+cells): ' + str(total_df.shape))

print('\033[1m\nDone! Your new raw datasets for Super Voxel based analysis are saved as separated csv files for each one of the ROIs you sleected.\033[0mYou can find them in ./voxelizer_final_datasets folder. If you want to anlayze them and check tuning run notebook 03c_Voxelizer_ANOVA_Nieder')

Loading experiment...

PROCESSING ROI 1

Voxelizing ROI 1... This could take time depending on your data and ROI size, but approx less than 20 min


Voxelizing chunks:   0%|          | 0/270 [00:00<?, ?it/s]